In [1]:
import pandas as pd
import numpy as np
import sys
from math import exp
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("pe_data/pe_users_13_14_15-2018-03-19T18.csv")
data
data2 = pd.read_csv("PE_test_data - Sheet1.csv")
data2



FileNotFoundError: File b'PE_test_data - Sheet1.csv' does not exist

In [ ]:
def data_storer(data):
    article_nums = np.unique(data['taskrun_article_number'])
    dict_of_article_df = dict()
    for i in article_nums:
        article =data[data['taskrun_article_number'] == i]
        question_nums = np.unique(article['question_number'])
        new_dict = dict()
        for x in question_nums:
            array = []
            array.append(article.loc[article['question_number']== x, 'question_text'][0:1])
            
            answers = [article.loc[article['question_number']== x, 'answer_number']]
            answers.append([article.loc[article['question_number']== x, 'contributor_id']])
            answers.append([article.loc[article['question_number']== x, 'start_pos']])
            answers.append([article.loc[article['question_number']== x, 'end_pos']])
            answers.append([article.loc[article['question_number']== x, 'source_text_length']])
            array.append(answers)

            new_dict[x] = array
            #this is where krippendorf goes
        dict_of_article_df[i] = new_dict
    return dict_of_article_df

In [ ]:
data1 = data_storer(data2)


In [ ]:
def get_question_text(data,article_num, question_num):
    return data[article_num][question_num][0]

In [ ]:
def get_question_answers(data, article_num, question_num):
    return data[article_num][question_num][1][0]

In [ ]:
def get_question_userid(data, article_num, question_num):
    return data[article_num][question_num][1][1][0]

In [ ]:
def get_question_start(data, article_num, question_num):
    return data[article_num][question_num][1][2][0]

In [ ]:
def get_question_end(data, article_num, question_num):
    return data[article_num][question_num][1][3][0]

In [ ]:
def get_text_length(data, article_num, question_num):
    return data[article_num][question_num][1][4][0]

In [ ]:
def cleanForUnitization(data, article_num, question_num):
    returnDict = dict()
    returnDict['Offsets'] = get_question_start(data, article_num,question_num).tolist()
    returnDict['Ends'] = get_question_end(data, article_num, question_num).tolist()
    returnDict['Categories'] = get_question_answers(data, article_num, question_num).tolist()
    returnDict['Raters'] = get_question_userid(data, article_num, question_num).tolist()
    return returnDict

In [ ]:
def get_num_users(data, article_num, question_num):
    return len(np.unique(get_question_userid(data, article_num, question_num)))

In [ ]:
def get_user_tuples(data, article_num, question_num):
    returnDict = dict()
    users = get_question_userid(data, article_num, question_num).tolist()
    starts = get_question_start(data, article_num, question_num).tolist()
    ends = get_question_end(data, article_num, question_num).tolist()
    index = 0
    for u in users:
        returnDict[u] = (starts[index], ends[index])
        index+= 1
    return returnDict

In [ ]:
def get_user_arrays(data, article_num, question_num):
    returnDict = dict()
    users = get_question_userid(data, article_num, question_num)
    answers = get_question_answers(data, article_num, question_num).tolist()
    index = 0
    for u in users:
        array = np.zeros(max(answers))
        array[answers[index]-1] = 1
        returnDict[u] = array
        index +=1
    return returnDict

    

In [ ]:
def get_question_answer_ratios(dictionary):
    users_num = len(dictionary.keys())
    returnArray = np.zeros(len(list(dictionary.values())[0]))
    for a in dictionary.values():
        returnArray = returnArray + a
    return (returnArray/users_num).tolist()

In [ ]:
def find_agreement_scores_u(data, article_num, question_num):
    user_array_dict = get_user_arrays(data, article_num, question_num)
    ans_ratio = get_question_answer_ratios(user_array_dict)
    num_users = get_num_users(data,article_num, question_num)
    grade = test_threshold_matrix(ans_ratio, num_users)
    if grade == 'L':
        print("agreement too low")
        return 
    elif grade == 'M':
        print("return to sender")
        return
    else:
        return find_starts_ends_of_winner(data,article_num,question_num,user_array_dict, ans_ratio)
    

In [ ]:
find_agreement_scores_u(data1, 1 ,3)

In [ ]:
def find_agreement_scores_c(data, article_num, question_num):
    user_array_dict = get_user_arrays(data, article_num, question_num)
    ans_ratio = get_question_answer_ratios(user_array_dict)
    num_users = get_num_users(data,article_num, question_num)
    grade = test_threshold_matrix(ans_ratio, num_users)
    if grade == 'L':
        print("agreement too low")
        return 
    elif grade == 'M':
        print("return to sender")
        return
    else:
        index = ans_ratio.index(max(ans_ratio))
        for k in user_array_dict.keys():
            if user_array_dict[k][index] == 0:
                del user_array_dict[k]
        return user_array_dict

In [ ]:
a = get_user_arrays(data1, 1, 3)
a



In [ ]:
b = get_question_answer_ratios(a)
b

In [ ]:
test_dict = cleanForUnitization(data1, 1, 3)
type(test_dict['Categories'][0])

In [ ]:
def test_threshold_matrix(array, num_of_users):
    percentage = max(array)
    kappa = 50 *( 1- exp(-num_of_users/15))
    omega = 1/1+exp(-kappa*(percentage -.5))
    deriv = (kappa * exp(-kappa*(percentage)))/(exp(-kappa*(percentage-.5))+1)**2
    if (deriv < 1):
        return 'M'
    elif (x>=.5):
        return 'H'
    else:
        return 'L'
    
    
    

In [ ]:
test_threshold_matrix(b, 3)

In [ ]:
get_user_tuples(data1, 1, 3)

In [ ]:
def find_starts_ends_of_winner(data, article_num, question_num, user_arrays, answer_ratio_array):
    user_start_end = get_user_tuples(data, article_num, question_num)
    winner_pos = answer_ratio_array.index(max(answer_ratio_array))
    for k in user_arrays.keys():
        if user_arrays[k][winner_pos] == 0:
            del user_start_end[k]
    return user_start_end

In [ ]:
find_starts_ends_of_winner(data1, 1, 3, a, b)

In [ ]:
type({1:1})

In [ ]:
def create_user_dataframe(data):
    """This function creates a DataFrame of User ID's, Reputation Scores, and number of questions answered."""
    data1 = pd.DataFrame(columns= ['Users', 'Score','Questions'] )
    for article_num in data.keys():
        for question_num in data[article_num].keys(): 
            users_q = get_question_userid(data, article_num, question_num)
            for ids in users_q:
                if ids not in data1.loc[:,'Users']:
                    data1 = data1.append({"Users":ids, "Score":5, "Questions": 1}, ignore_index = True)
    return data1

In [ ]:
def do_the_calculation_nominal(userID, answers, answer_choice, data):
    """Using the same dataframe of userIDs, rep scores, and number of questions, changes the vals of the dataframe
    such that if the user in the list of USERID gets their answer right, they add 1 to their score, and 0 if they are
    wrong."""
    tups = list()
    for ids in userID:
        for a in answers:
            pair = (ids, a)
            if pair not in tups:
                tups.append(pair)
    for t in tups:
        user = t[0]
        answer = t[1]
        if (answer == answer_choice):
            do_math(data, user, 1)
        else:
            do_math(data, user, 0)

In [ ]:
def do_the_calculation_ordinal(userID, answers, answer_aggregated,num_of_choices, data):
    """Using the same dataframe of userIDs, rep scores, and number of questions, changes the vals of the dataframe
    such that the they recieve the distance from the average answer chosen as a ratio of 0 to 1, 
    and that is added to their rep score."""
    tups = list()
    for ids in userID:
        for a in answers:
            pair = (ids, a)
            if pair not in tups:
                tups.append(pair)
    answer_choice = np.mean(answer_aggregated)
    for t in tups:
        user = t[0]
        answer = t[1]
        points = (1 - abs(answer_choice - answer)/num_of_choices)
        do_math(data, user, points)

In [ ]:
def do_math(data, userID, reward):
    """This function takes in the points added to one user and changes the dataframe to update that one user's score
    using the equations set for calculating reputation."""
    user = data.loc[data['Users']== userID]
    r = user['Score']
    n = user['Questions']
    q_score = 10* (1 - exp(-n))
    points = r * n / q_score
    points = points + reward
    n = n+1
    q_score = 10* (1 - exp(-n))
    user['Questions'] = n
    user['Score'] = (points /n) * q_score


In [ ]:
def calc_influence(data, userID):
    """Taking in a list of UserID's, this will take the repuation score of each User and output a list of their influence
    based on their reputation score."""
    return_vals = list()
    for u in userID:
        r = data.loc[data['Users']== u]['Score']
        inf = 2/ (1 + exp(-r + 5))
        return_vals.append(inf)
    return return_vals

In [ ]:
def userid_to_CSV(dataframe):
    """This function will save the User Rep Score dataframe as UserRepScores.csv"""
    dataframe.to_csv("UserRepScores")

    

In [ ]:
def CSV_to_userid(path):
    """This function will take in the path name of the UserRepScore.csv and output the dataframe corresponding."""
    return pd.read_csv(path, index_col = False).loc[:,['Users','Score','Questions']]

In [ ]:
i = create_user_dataframe(data1)
userid_to_CSV(i)
CSV_to_userid("UserRepScores")
i.loc[i['Users'] == 1,"Score"] = 4
change_user_score(i, 1, 5)
calc_influence(i, [1])

## 1. Function that takes in raw data and splits by article number into a dict of DF.
2. Function that takes in DF of 1 article and outputs a dict

In [ ]:
dataTemp = data_storer(data2)
get_question_answers(dataTemp, 1722, 3)

In [ ]:
data3 = pd.read_csv("question_scores.csv")
data3

In [ ]:
np.count_nonzero(data3['Agreement Score'])

In [ ]:
data3

In [ ]:
data4 = data3[data3['Agreement Score'] != "2Short loozerz"].dropna().astype('float64')


## data3['Agreement Score'].dropna().hist()
plt.show()

In [ ]:
ord_data = data4[data4['Question Number'].isin([1, 8, 9, 12, 13])]
nom_data =  data4[data4['Question Number'].isin([5,10])]
int_data =  data4[data4['Question Number'].isin([7,11])]
mult_data =  data4[data4['Question Number'].isin([6])]

In [ ]:
ord_data['Agreement Score'].hist()

plt.show()

In [ ]:
nom_data['Agreement Score'].hist()
plt.show()

In [ ]:
int_data['Agreement Score'].hist()
plt.show()

In [ ]:
mult_data['Agreement Score'].hist()
plt.show()

In [13]:
ex = pd.DataFrame(columns= ['users','index'] +list(range(30)))
ex = ex.append({"users":1, 'index':1}, ignore_index=True)

/Users/JoshuaChung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  


In [16]:
ex.loc[ex['users']==1,1] = 1
ex.loc[ex['users']==1,2] = 1
ex.loc[ex['users']==1,3] = 1
ex.loc[ex['users']==1,4] = 1

In [29]:
ex.loc[ex['users']==1, range(30)]


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
np.mean(np.array(ex.loc[ex['users']==1, range(30)].dropna(axis=1)))

1.0